## Decision Trees

This notebook contains the implementation of a decision tree using the following criterion:

1. Chi-square
<br>
2. Gini Impurity
<br>
3. Number of correct guesses (provided in class)

In [1]:
import numpy as np

# load the csv
csv = np.genfromtxt('movie.csv', delimiter=',')

# remove header
csv = csv[1:]

# convert ratings from +2, +1, 0, -1, -2 to 0 to 2 for like
# and -1 to -2 for dislike
base_ratings = csv[:, 0]
base_ratings[base_ratings >= 0] = 1
base_ratings[base_ratings < 0] = -1

# shuffle data
np.random.shuffle(csv)

# split data 80/20 training/test
training_size = int(csv.shape[0] * 0.8)
training_data = csv[:training_size]
test_data = csv[training_size:]

# separate dataset into the label and the feature values
# training_data = csv
ratings = np.array(training_data[:, 0])
examples = np.array(training_data[:, 1:])

In [2]:
# tree data structure
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def isLeaf(self):
        return self.right == None and self.left == None

    def height(self):
        if self.isLeaf():
            return 1
        return 1 + max(self.left.height(), self.right.height())
    
    def inorder(self):
        if self.left is not None:
            self.left.inorder()
        print(self.value)
        if self.right is not None:
            self.right.inorder()

In [3]:
# from https://stackoverflow.com/questions/34012886/print-binary-tree-level-by-level-in-python
def print_tree(root, val="val", left="left", right="right"):
    def display(root, val=val, left=left, right=right):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if getattr(root, right) is None and getattr(root, left) is None:
            line = str(root.value)
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if getattr(root, right) is None:
            lines, n, p, x = display(getattr(root, left))
            s = str(root.value)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if getattr(root, left) is None:
            lines, n, p, x = display(getattr(root, right))
            s = str(root.value)
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = display(getattr(root, left))
        right, m, q, y = display(getattr(root, right))
        s = str(root.value)
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

    lines, *_ = display(root, val, left, right)
    for line in lines:
        print(line)

In [4]:
# prediction function
def predict(tree, x):
    if tree.isLeaf(): # if leaf, value must be the prediction
        return tree.value

    feature = tree.value # else, value is a feature to query on

    # if the feature to query on is false, predict with left tree
    if not x[feature]:
        return predict(tree.left, x)
    else: # else, predict with right
        return predict(tree.right, x)

## Chi-square

Following the ideas presented [in this resource](https://www.analyticsvidhya.com/blog/2021/03/how-to-select-best-split-in-decision-trees-using-chi-square/), I implement a decision tree with chi-square as the splitting criterion

In [5]:
print("Ratio of 1 to 0 values of the original dataset")
print("1 counts:", sum(base_ratings == 1))
print("0 counts:", sum(base_ratings == -1))

Ratio of 1 to 0 values of the original dataset
1 counts: 12
0 counts: 8


In [18]:
def chi_square(actual, expected):
    if expected == 0:
        expected = 1
    return np.sqrt(((actual - expected) ** 2) / expected)

# training function
def train_chi(x, y, features):
        
        y_sum = np.sum(y) # take summation of y

        if y_sum == 0: # if our sum is 0, then they're 50/50 positive/negative labels
            y_sum = 1 # arbitrarily set it to positive

        guess = np.sign(y_sum) # take the sign of y_sum, majority label will win in this case

        if np.all(y == 1) or np.all(y == -1): # if labels were all 1 or all -1, there's no other guess
            return TreeNode(guess)

        if not features: # there are no more features to query on
            return TreeNode(guess)

        # calculate percentage to be used in getting expected value
        no_percentage =  sum(y == 0) / y.shape[0]
        yes_percentage =  sum(y == 1) / y.shape[0]
        
        chi_square_val = {}
        for feature in features:
            # split examples according to x[feature] is 0 or 1
            x_no = x[x[:, feature] == 0]
            x_yes = x[x[:, feature] == 1]

            # split label according to x[feature] being 0 or 1
            y_no = y[x[:, feature] == 0]
            y_yes = y[x[:, feature] == 1]
            
            # getting split+label pairs
            expected_no_no = x_no.shape[0] * no_percentage
            actual_no_no = sum(y_no == 0)
            
            expected_no_yes = x_no.shape[0] * no_percentage
            actual_no_yes = sum(y_no == 1)

            expected_yes_no = x_yes.shape[0] * yes_percentage
            actual_yes_no = sum(y_yes == 0)
            
            expected_yes_yes = x_yes.shape[0] * yes_percentage
            actual_yes_yes = sum(y_yes == 1)
            
            # calculating chi square value for each split+label pair
            no_no_cs = chi_square(expected_no_no, actual_no_no)
            no_yes_cs = chi_square(expected_no_yes, actual_no_yes)
            yes_no_cs = chi_square(expected_yes_no, actual_yes_no)
            yes_yes_cs = chi_square(expected_yes_yes, actual_yes_yes)

            # getting feature chi square value
            chi_square_val[feature] = no_no_cs + no_yes_cs + yes_no_cs + yes_yes_cs

        best_feature = max(chi_square_val, key=chi_square_val.get) # take feature with the best score
        remaining_features = [feature for feature in features if feature != best_feature] # new list of features, less the best feature
        
        # split examples and labels where best_feature is 0 or 1
        x_best_no = x[x[:, best_feature] == 0]
        y_best_no = y[x[:, best_feature] == 0]

        x_best_yes = x[x[:, best_feature] == 1]
        y_best_yes = y[x[:, best_feature] == 1]

        if len(remaining_features) == 0:
            return TreeNode(1)
        
        left = train_chi(x_best_no, y_best_no, remaining_features)
        right = train_chi(x_best_yes, y_best_yes, remaining_features)
        return TreeNode(best_feature, left, right)

In [21]:
# train the tree
tree = train_chi(examples, ratings, list(range(csv.shape[1]-1)))
print("--------PRINTING TREE---------")
print_tree(tree)
print()

# score on test data
x_test = test_data[:, 1:]
y_test = test_data[:, 0]

score = 0
for x, y in zip(x_test, y_test):
    # if we get a +1 it means we got the same sign, it's a correct prediction
    if predict(tree, x) * y > 0:
        score += 1
print("SCORE IS: ", score / test_data.shape[0]) # print out score

--------PRINTING TREE---------
     ___1_____    
    /         \   
   _2_       _2_  
  /   \     /   \ 
-1.0 1.0   _3  1.0
          /  \    
          4  1    
         / \      
         1 1      

SCORE IS:  1.0


## Gini Impurity

This time, we implement a DTree using gini impurity, one of the most common splitting criterion

In [22]:
def get_weighted_gini_impurity(pair_no, pair_yes, yes_shape, no_shape):
    
    def gini_index(val_yes, val_no):
        denominator = val_yes + val_no
        if denominator == 0:
            denominator = 1
        return 1 - ((val_yes/(denominator))**2 + (val_no/(denominator))**2)

    weighted_gini_impurity = lambda index_yes, index_no, yes, no: (index_yes*(yes/(yes+no))) + (index_no*(no/(yes+no)))

    # get gini index for each label
    no_gini_index = gini_index(pair_no[0], pair_no[1])
    yes_gini_index = gini_index(pair_yes[0], pair_yes[1])
    
    return weighted_gini_impurity(yes_gini_index, no_gini_index, yes_shape, no_shape)
    

# training function
def train_gini(x, y, features):
        
        y_sum = np.sum(y) # take summation of y

        if y_sum == 0: # if our sum is 0, then they're 50/50 positive/negative labels
            y_sum = 1 # arbitrarily set it to positive

        guess = np.sign(y_sum) # take the sign of y_sum, majority label will win in this case

        if np.all(y == 1) or np.all(y == -1): # if labels were all 1 or all -1, there's no other guess
            return TreeNode(guess)

        if not features: # there are no more features to query on
            return TreeNode(guess)

        impurity = {}
        for feature in features:
#             # split examples according to x[feature] is 0 or 1
            x_no = x[x[:, feature] == 0]
            x_yes = x[x[:, feature] == 1]

            # split label according to x[feature] being 0 or 1
            y_no = y[x[:, feature] == 0]
            y_yes = y[x[:, feature] == 1]

            # split labels according to whether they are 1 or 0
            y_no_yes = y_no[y_no > 0].shape[0]
            y_no_no = y_no[y_no < 0].shape[0]
            y_yes_yes = y_yes[y_yes > 0].shape[0]
            y_yes_no = y_yes[y_yes < 0].shape[0]            

            # score for this feature is our total score
            impurity[feature] = get_weighted_gini_impurity([y_no_yes, y_no_no], [y_yes_yes, y_yes_no], y_yes.shape[0], y_no.shape[0])

        best_feature = min(impurity, key=impurity.get) # take feature with the best score
        remaining_features = [feature for feature in features if feature != best_feature] # new list of features, less the best feature
        
        # split examples and labels where best_feature is 0 or 1
        x_best_no = x[x[:, best_feature] == 0]
        y_best_no = y[x[:, best_feature] == 0]

        x_best_yes = x[x[:, best_feature] == 1]
        y_best_yes = y[x[:, best_feature] == 1]
        
        if len(remaining_features) == 0:
            return TreeNode(1)
        
        # train left child
        left = train_gini(x_best_no, y_best_no, remaining_features)
        # train right child
        right = train_gini(x_best_yes, y_best_yes, remaining_features)
        return TreeNode(best_feature, left, right)

In [23]:
# train the tree
tree = train_gini(examples, ratings, list(range(csv.shape[1]-1)))
print("--------PRINTING TREE---------")
print_tree(tree)
print()

# make a prediction
x = [0, 1, 1, 1, 1]
# print(predict(tree, x)) # print out prediction

# score on test data
x_test = test_data[:, 1:]
y_test = test_data[:, 0]

score = 0
for x, y in zip(x_test, y_test):
    # if we get a +1 it means we got the same sign, it's a correct prediction
    if predict(tree, x) * y > 0:
        score += 1
print("SCORE IS: ", score / test_data.shape[0]) # print out score

--------PRINTING TREE---------
     ________2_  
    /          \ 
   _1____     1.0
  /      \       
-1.0    _0_      
       /   \     
     -1.0  3     
          / \    
          1 1    

SCORE IS:  0.75


## Count of Correct Guesses

This implementation of a DTree is given in the class, with the count of correct guesses as the splitting criterion

In [24]:
def train(x, y, features):
        y_sum = np.sum(y) # take summation of y

        if y_sum == 0: # if our sum is 0, then they're 50/50 positive/negative labels
            y_sum = 1 # arbitrarily set it to positive

        guess = np.sign(y_sum) # take the sign of y_sum, majority label will win in this case

        if np.all(y == 1) or np.all(y == -1): # if labels were all 1 or all -1, there's no other guess
            return TreeNode(guess)

        if not features: # there are no more features to query on
            return TreeNode(guess)

        score = {}
        for feature in features:
            # split examples according to x[feature] is 0 or 1
            x_no = x[x[:, feature] == 0]
            x_yes = x[x[:, feature] == 1]

            # split label according to x[feature] being 0 or 1
            y_no = y[x[:, feature] == 0]
            y_yes = y[x[:, feature] == 1]

            # take majority label as guess
            guess_no = np.sign(np.sum(y_no))
            guess_yes = np.sign(np.sum(y_yes))

            # count number of ratings examples we would get right with our guesses
            score_no = np.sum(y_no == guess_no)
            score_yes = np.sum(y_yes == guess_yes)

            # score for this feature is our total score
            score[feature] = score_no + score_yes

        best_feature = max(score, key=score.get) # take feature with the best score
        remaining_features = [feature for feature in features if feature != best_feature] # new list of features, less the best feature

        # split examples and labels where best_feature is 0 or 1
        x_best_no = x[x[:, best_feature] == 0]
        y_best_no = y[x[:, best_feature] == 0]

        x_best_yes = x[x[:, best_feature] == 1]
        y_best_yes = y[x[:, best_feature] == 1]

        left = train(x_best_no, y_best_no, remaining_features)
        right = train(x_best_yes, y_best_yes, remaining_features)
        return TreeNode(best_feature, left, right)

In [25]:
# train the tree
tree = train(examples, ratings, list(range(csv.shape[1]-1)))
print("--------PRINTING TREE---------")
print_tree(tree)
print()

# make a prediction
x = [0, 1, 1, 1, 1]
# print(predict(tree, x)) # print out prediction

# score on test data
x_test = test_data[:, 1:]
y_test = test_data[:, 0]

score = 0
for x, y in zip(x_test, y_test):
    # if we get a +1 it means we got the same sign, it's a correct prediction
    if predict(tree, x) * y > 0:
        score += 1
print("SCORE IS: ", score / test_data.shape[0]) # print out score

--------PRINTING TREE---------
     _____________2_  
    /               \ 
   _0________      1.0
  /          \        
-1.0        _3__      
           /    \     
          _4  -1.0    
         /  \         
        _1  1         
       /  \           
     -1.0 1           

SCORE IS:  0.75
